In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import neo
import neo_importers.neo_spike_importer as spike_importer
from neo_importers.neo_wrapper import MNGRecording
from features import FeatureDatabase

file_name = Path("..")/".."/"Files"/"latency_experiment_roberto"/"20_05_13_U1a_pulse_Latenz.smr"

# maybe we could make the spike importer return a recording object immediately?
# or recording + block representation?
bl, id_map = spike_importer.import_spike_file(file_name, 
                                      stimuli_event_channels={"DigMark"},
                                      action_potential_channels= "all"
                                      )
seg: neo.core.Segment = bl.segments[0]
recording = MNGRecording(bl.segments[0])

print("There are the following channels in this recording: " + str(recording.all_channels.keys()))

In [ ]:
print(recording.electrical_stimulus_channels_raw["es.0"].array_annotations)

In [ ]:
from neo.core import AnalogSignal

rd_channel: AnalogSignal
for key, rd_channel in recording.raw_data_channels.items():
    print(key, rd_channel.name)

In [ ]:
%autoreload 2
from features import ResponseLatencyFeatureExtractor, NormalizedSignalEnergyExtractor, SpikeCountExtractor, AdaptiveSpikeCountExtractor
from quantities import second

# define feature db
db = FeatureDatabase(Path("../../Files/latency_experiment_roberto/20_05_13_U1a_pulse_Latenz_Features"), recording)

try: # check if we can just load the features somehow
    db.load()
    print("Found feature database and loaded from files...")
except Exception as ex: # extract the features from the data base
    ap_channels = list(recording.action_potential_channels.keys())
    db.extract_features(ap_channels, ResponseLatencyFeatureExtractor, stimulus_channel = "es.0")
    db.extract_features(ap_channels, NormalizedSignalEnergyExtractor)
    db.extract_features(ap_channels, SpikeCountExtractor, timeframe = 100 * second, num_intervals = 8)
    db.extract_features(ap_channels, AdaptiveSpikeCountExtractor, timeframe = 100 * second, num_intervals = 6)
    db.store()
    print("Created and stored new feature db...")

In [ ]:
%autoreload 2
from fibre_tracking import APTrack

raw_signal = recording.raw_data_channel_by_name(name = "Signal")
el_stimuli = recording.electrical_stimulus_channels["es.0"]
ap_channel = recording.action_potential_channels["ap.2"]

ap_track = APTrack.from_aps(el_stimuli = el_stimuli, aps = list(ap_channel[0 : 3]))
ap_track.color = "green"

print(ap_track)

In [ ]:
%autoreload 2
ap_track.extend_downwards(raw_signal = raw_signal, el_stimuli = el_stimuli, num_sweeps = 2)

In [ ]:
%autoreload 2
from plotting import FallingLeafPlot
from quantities import second, minute
from neo_importers.neo_wrapper import TypeID

signal_channel = id_map[TypeID.RAW_DATA]["Signal"]
flp = FallingLeafPlot(recording = recording, feature_db = db, \
                      width = 750, height = 450)

flp.plot(action_potential_channels = "all", plot_raw_signal = True, analog_signal_channel = signal_channel, \
         t_start = 0 * minute, post_stimulus_timeframe = 0.01 * second, num_intervals = 25, 
         ap_tracks = [ap_track])